# Phase 2: Overfitting Experiment (Fixed Version)

**Goal**: Train TinyLlama to memorize 10 examples perfectly

**Expected**: Model learns to generate valid JSON for trained examples

**Success Criteria**: 80%+ valid JSON on training examples

**Fixes Applied**:
- Consolidated all installs and imports (no repetitions).
- Uninstalled TensorFlow to prevent conflicts.
- Fully disabled WandB with environment variables.
- Added GPU verification cell.
- Used 4-bit quantization for memory efficiency.
- Reduced batch size and added gradient accumulation to avoid OOM.
- Standardized dataset path and added error handling.
- Improved generation with max_new_tokens and better response extraction.
- Removed unnecessary code (e.g., old fixes for model_utils.py - assume it's already fixed).

**Instructions**: Run cells in order on T4 GPU. Use keep-alive JS in console: function KeepClicking(){document.querySelector("colab-connect-button").click()}setInterval(KeepClicking,60000)

In [ ]:
# Cell 1: Clean Installs and Setup (Consolidated, No Conflicts)
!pip uninstall -y tensorflow  # Remove TF to prevent interference
!pip install --upgrade transformers torch accelerate bitsandbytes peft datasets trl -q

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Disable WandB fully (prevents prompts and disconnections)
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"

print("✅ Installs complete and WandB disabled!")

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.0 MB/s eta 0:00:00
Mounted at /content/drive
✅ Installs complete and WandB disabled!


In [ ]:
# Cell 2: Copy Files and Verify (Standardized Paths)
import os
os.chdir('/content')

# Remove old folder if exists
!rm -rf fine_tuning

# Copy fine_tuning folder and dataset (use your exact Drive path)
!cp -r "/content/drive/MyDrive/fine_tuning" .
!cp "/content/drive/MyDrive/fine_tuning/rpg_training_dataset_gpt4_1_filtered.jsonl" .

# Verify files
!ls -la fine_tuning/
print("✅ Files copied!")

total 920
drwx------ 6 root root   4096 Sep  1 03:40 .
drwxr-xr-x 1 root root   4096 Sep  1 03:40 ..
drwx------ 2 root root   4096 Sep  1 03:40 colab_notebooks
drwx------ 2 root root   4096 Sep  1 03:40 phase_1_pretrain_test
drwx------ 2 root root   4096 Sep  1 03:40 phase_2_overfitting
-rw------- 1 root root 915819 Sep  1 03:40 rpg_training_dataset_gpt4_1_filtered.jsonl
drwx------ 2 root root   4096 Sep  1 03:40 utils
✅ Files copied!


In [ ]:
# Cell 3: Imports (Consolidated, No Duplicates)
import sys
sys.path.append('/content/fine_tuning')

from utils.data_utils import load_dataset
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import torch

print("✅ All imports loaded!")

⚙️  Running in WANDB offline mode
✅ All imports loaded!


In [ ]:
# Cell 4: GPU Verification (Run This First!)
print("Checking GPU...")
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
!nvidia-smi  # Detailed GPU info

if not torch.cuda.is_available():
    print("❌ WARNING: No GPU detected! Change runtime to T4 GPU and restart.")

Checking GPU...
GPU Available: True
GPU Name: Tesla T4
Mon Sep  1 03:40:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                 

In [ ]:
# Cell 5: Load Dataset and Prepare Tiny Version
print("📊 Loading dataset...")
dataset = load_dataset("rpg_training_dataset_gpt4_1_filtered.jsonl")
print(f"✅ Loaded {len(dataset)} examples")

# Create tiny dataset (10 examples)
tiny_dataset = dataset[:10]

# Format for training
formatted_data = []
for item in tiny_dataset:
    prompt = item['prompt']
    response = json.dumps(item['response'])
    formatted = f"<|user|>\n{prompt}<|end|>\n<|assistant|>\n{response}<|end|>"
    formatted_data.append({"text": formatted})

print(f"✅ Created {len(formatted_data)} training examples")
print(f"📋 Sample: {formatted_data[0]['text'][:200]}...")

📊 Loading dataset...
✅ Loaded 421 examples
✅ Created 10 training examples
📋 Sample: <|user|>
Generate a tilemap for a game level, where all the edges should be walls
there should only be *ONE* player and multiple enemies, all enemies should be placed
randomly and the player should be...


In [ ]:
# Cell 6: Setup Model and LoRA (With Quantization for Memory)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print(f"🤖 Loading {model_name} for training...")

# Quantization for low memory
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

# Load tokenizer and model
train_tokenizer = AutoTokenizer.from_pretrained(model_name)
train_tokenizer.pad_token = train_tokenizer.eos_token

train_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)

print("✅ Model loaded with quantization!")

# Setup LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05
)

train_model = get_peft_model(train_model, lora_config)
train_model.print_trainable_parameters()

print("✅ LoRA setup complete!")

🤖 Loading TinyLlama/TinyLlama-1.1B-Chat-v1.0 for training...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded with quantization!
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
✅ LoRA setup complete!


In [ ]:
# Cell 7: Tokenize Data
train_dataset = Dataset.from_list(formatted_data)

def tokenize_function(examples):
    tokenized = train_tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=1024
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

print(f"✅ Tokenized {len(tokenized_dataset)} examples")
print(f"📋 Sample keys: {list(tokenized_dataset[0].keys())}")

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ Tokenized 10 examples
📋 Sample keys: ['text', 'input_ids', 'attention_mask', 'labels']


In [ ]:
# Cell 8: Train (Optimized for Stability)
training_args = TrainingArguments(
    output_dir="/content/overfit_model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=1,  # Reduced for memory
    gradient_accumulation_steps=2,  # Simulates batch=2
    learning_rate=2e-4,
    logging_steps=1,
    save_steps=100,
    warmup_steps=0,
    fp16=True,
    dataloader_num_workers=0,
    report_to=None
)

trainer = Trainer(
    model=train_model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("🏋️ Starting overfitting training...")
print("📊 Watch the loss decrease (sign of memorization)")

trainer.train()

print("✅ Overfitting training complete!")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🏋️ Starting overfitting training...
📊 Watch the loss decrease (sign of memorization)


Step,Training Loss
1,0.382500
2,0.360400
3,0.344000
4,0.323900
5,0.305600
6,0.287900
7,0.272200
8,0.258100
9,0.242700
10,0.228800


✅ Overfitting training complete!


In [ ]:
# Cell 9: Save Model
save_path = "/content/drive/MyDrive/overfit_tinyllama_rpg"
train_model.save_pretrained(save_path)
train_tokenizer.save_pretrained(save_path)
print(f"💾 Model saved to: {save_path}")

💾 Model saved to: /content/drive/MyDrive/overfit_tinyllama_rpg


In [ ]:
# Cell 10: Test Overfitted Model
# Fix pad token for attention mask warning
train_tokenizer.pad_token = train_tokenizer.convert_tokens_to_ids("<PAD>")  # Use a custom pad token
if train_tokenizer.pad_token is None:
    train_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    train_model.resize_token_embeddings(len(train_tokenizer))

train_model.eval()
test_results = []

print("🧪 Re-testing with length fixes...")

for i, example in enumerate(tiny_dataset[:5]):
    prompt = f"<|user|>\n{example['prompt']}<|end|>\n<|assistant|>\n"

    inputs = train_tokenizer(prompt, return_tensors="pt", padding=True).to(train_model.device)

    with torch.no_grad():
        outputs = train_model.generate(
            **inputs,  # Pass tokenized dict
            max_new_tokens=4096,  # Larger for full JSON
            temperature=0.1,
            do_sample=False,
            pad_token_id=train_tokenizer.pad_token_id,
            eos_token_id=train_tokenizer.eos_token_id
        )

    response = train_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    # Debug: Print full response
    print(f"Full Response {i+1}:\n{response}\n---")

    try:
        parsed_json = json.loads(response)
        required_fields = ['width', 'height', 'walls', 'enemies', 'player_pos']
        has_all_fields = all(field in parsed_json for field in required_fields)

        if has_all_fields and isinstance(parsed_json['walls'], list) and len(parsed_json['walls']) > 0:
            result = "✅ PERFECT"
            test_results.append(True)
        else:
            result = "⚠️ PARTIAL"
            test_results.append(False)
        print(f"Test {i+1}: {result}")
    except json.JSONDecodeError as e:
        result = f"❌ FAILED (Error: {str(e)})"
        test_results.append(False)
        print(f"Test {i+1}: {result}")

success_rate = sum(test_results) / len(test_results)
print(f"📊 SUCCESS RATE: {success_rate:.1%}")

if success_rate >= 0.8:
    print("🎉 SUCCESS: Ready for full fine-tuning!")
elif success_rate > 0:
    print("⚠️ Good start—try shortening data or more training.")
else:
    print("❌ Still truncated—check full responses above.")

NameError: name 'train_tokenizer' is not defined

# Standalone Test: Load Saved Model and Run Overfitting Tests

**Note**: This loads your trained model from Drive without re-training. Run on T4 GPU.
Use keep-alive JS in Console to prevent timeouts.

In [ ]:
# Minimal installs (only what's needed for loading/testing)
!pip install transformers torch accelerate bitsandbytes peft datasets -q

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
from datasets import Dataset

print("✅ Setup complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.3 MB/s eta 0:00:00
Mounted at /content/drive
✅ Setup complete!


In [ ]:
# Load your dataset (same as before, for tiny_dataset)
def load_dataset(file_path):
    dataset = []
    with open(file_path, 'r') as f:
        for line in f:
            dataset.append(json.loads(line))
    return dataset

dataset_path = "/content/drive/MyDrive/fine_tuning/rpg_training_dataset_gpt4_1_filtered.jsonl"  # Adjust if needed
dataset = load_dataset(dataset_path)
tiny_dataset = dataset[:10]  # Same 10 examples used for training

print(f"✅ Loaded dataset and created tiny_dataset with {len(tiny_dataset)} examples")

✅ Loaded dataset and created tiny_dataset with 10 examples


In [ ]:
# Path to your saved model
save_path = "/content/drive/MyDrive/overfit_tinyllama_rpg"

# Load base model with quantization (for memory)
from transformers import BitsAndBytesConfig
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
train_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load LoRA adapters from saved path
train_model = PeftModel.from_pretrained(train_model, save_path)

# Load tokenizer
train_tokenizer = AutoTokenizer.from_pretrained(save_path)

# Fix pad token properly (add if missing, set as string)
pad_token = "[PAD]"
if pad_token not in train_tokenizer.get_vocab():
    train_tokenizer.add_special_tokens({'pad_token': pad_token})
    train_model.resize_token_embeddings(len(train_tokenizer))
    print(f"✅ Added custom pad token: {pad_token}")

# Set pad_token to the string (fixes the error)
train_tokenizer.pad_token = pad_token

# Optional: Set pad_token_id (the ID) for reference
train_tokenizer.pad_token_id = train_tokenizer.convert_tokens_to_ids(pad_token)

print("✅ Saved model and tokenizer loaded successfully! Pad token fixed.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


✅ Added custom pad token: [PAD]
✅ Saved model and tokenizer loaded successfully! Pad token fixed.


In [ ]:
train_model.eval()
test_results = []

print("🧪 Testing loaded model...")

for i, example in enumerate(tiny_dataset[:5]):
    prompt = f"<|user|>\n{example['prompt']}<|end|>\n<|assistant|>\n"

    inputs = train_tokenizer(prompt, return_tensors="pt", padding=True).to(train_model.device)

    with torch.no_grad():
        outputs = train_model.generate(
            **inputs,
            max_new_tokens=4096,  # Large enough for full JSON
            temperature=0.1,
            do_sample=False,
            pad_token_id=train_tokenizer.pad_token_id,
            eos_token_id=train_tokenizer.eos_token_id
        )

    response = train_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    # Debug: Print full response
    print(f"Full Response {i+1} (Length: {len(response)} chars):\n{response}\n---")

    try:
        parsed_json = json.loads(response)
        required_fields = ['width', 'height', 'walls', 'enemies', 'player_pos']
        has_all_fields = all(field in parsed_json for field in required_fields)

        if has_all_fields and isinstance(parsed_json['walls'], list) and len(parsed_json['walls']) > 0:
            result = "✅ PERFECT"
            test_results.append(True)
        else:
            result = "⚠️ PARTIAL"
            test_results.append(False)
        print(f"Test {i+1}: {result}")
    except json.JSONDecodeError as e:
        result = f"❌ FAILED (Error: {str(e)})"
        test_results.append(False)
        print(f"Test {i+1}: {result}")

success_rate = sum(test_results) / len(test_results)
print(f"📊 SUCCESS RATE: {success_rate:.1%}")

if success_rate >= 0.8:
    print("🎉 SUCCESS: Ready for full fine-tuning!")
elif success_rate > 0:
    print("⚠️ Good start—try more training if needed.")
else:
    print("❌ Still issues—check full responses for truncation.")

In [ ]:
!pip install json-repair -q
import json_repair
print("✅ JSON repair installed!")

✅ JSON repair installed!


In [ ]:
import re

train_model.eval()
test_results = []

print("🧪 Testing with improved cleaning...")

for i, example in enumerate(tiny_dataset[:5]):
    prompt = f"<|user|>\n{example['prompt']}<|end|>\n<|assistant|>\n"

    inputs = train_tokenizer(prompt, return_tensors="pt", padding=True).to(train_model.device)

    with torch.no_grad():
        outputs = train_model.generate(
            **inputs,
            max_new_tokens=4096,
            temperature=0.1,
            do_sample=False,
            pad_token_id=train_tokenizer.pad_token_id,
            eos_token_id=train_tokenizer.eos_token_id
        )

    response = train_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    # Step 1: Extract potential JSON block
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    if json_match:
        potential_json = json_match.group(0)
    else:
        potential_json = response

    # Step 2: Repair and parse (fixes small errors like missing quotes)
    try:
        cleaned_response = json_repair.loads(potential_json)  # Repairs and parses to dict
        # Convert back to string for consistency
        cleaned_response_str = json.dumps(cleaned_response)

        # Validate fields
        required_fields = ['width', 'height', 'walls', 'enemies', 'player_pos']
        has_all_fields = all(field in cleaned_response for field in required_fields)

        if has_all_fields and isinstance(cleaned_response['walls'], list) and len(cleaned_response['walls']) > 0:
            result = "✅ PERFECT"
            test_results.append(True)
        else:
            result = "⚠️ PARTIAL"
            test_results.append(False)
        print(f"Test {i+1}: {result}")
        print(f"Cleaned Response Preview: {cleaned_response_str[:150]}...\n")
    except Exception as e:
        result = f"❌ FAILED (Error: {str(e)})"
        test_results.append(False)
        print(f"Test {i+1}: {result}")
        print(f"Potential JSON Preview: {potential_json[:150]}...\n")

    print("---")

success_rate = sum(test_results) / len(test_results)
print(f"📊 SUCCESS RATE: {success_rate:.1%}")

if success_rate >= 0.8:
    print("🎉 SUCCESS: Phase 2 complete!")
elif success_rate > 0:
    print("⚠️ Partial—review cleaned responses.")
else:
    print("❌ Repair failed—may need re-training with stop tokens.")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧪 Testing with improved cleaning...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 1: ⚠️ PARTIAL
Cleaned Response Preview: {"width": 20, "height": 15, "walls": [{"x": 0, "y": 0}, {"x": 1, "y": 0}, {"x": 2, "y": 0}, {"x": 3, "y": 0}, {"x": 4, "y": 0}, {"x": 5, "y": 0}, {"x"...

---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 2: ⚠️ PARTIAL
Cleaned Response Preview: {"width": 20, "height": 15, "walls": [{"x": 0, "y": 0}, {"x": 1, "y": 0}, {"x": 2, "y": 0}, {"x": 3, "y": 0}, {"x": 4, "y": 0}, {"x": 5, "y": 0}, {"x"...

---


KeyboardInterrupt: 